In [1]:
import pandas as pd
d = {"headers":{"Employee":["id","month","salary"]},"rows":{"Employee":[[1,1,20],[2,1,20],[1,2,30],[2,2,30],[3,2,40],[1,3,40],[3,3,60],[1,4,60],[3,4,70],[1,7,90],[1,8,90]]}}
pd.DataFrame(d['rows']['Employee'], columns=d['headers']['Employee']).to_csv("./Employee.txt", index=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("find cumulative salary of an employee")\
                    .config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [10]:
e = spark.read.option("header", True).csv("./Employee.txt")
e =e.withColumn("salary", e.salary.cast('int')).withColumn("id", e.id.cast('int')).withColumn("month", e.month.cast('int'))
e.show()

+---+-----+------+
| id|month|salary|
+---+-----+------+
|  1|    1|    20|
|  2|    1|    20|
|  1|    2|    30|
|  2|    2|    30|
|  3|    2|    40|
|  1|    3|    40|
|  3|    3|    60|
|  1|    4|    60|
|  3|    4|    70|
|  1|    7|    90|
|  1|    8|    90|
+---+-----+------+



Write an SQL query to calculate the cumulative salary summary for every employee in a single unified table.

The cumulative salary summary for an employee can be calculated as follows:

For each month that the employee worked, sum up the salaries in that month and the previous two months. This is their 3-month sum for that month. If an employee did not work for the company in previous months, their effective salary for those months is 0.
Do not include the 3-month sum for the most recent month that the employee worked for in the summary.
Do not include the 3-month sum for any month the employee did not work.
Return the result table ordered by id in ascending order. In case of a tie, order it by month in descending order.

The query result format is in the following example.

In [22]:
def addAlias(df, n):
    for i in df.columns:
        df = df.withColumnRenamed(i, f'{i}_{n}')
    return df
e1 = addAlias(e,1)
e1.printSchema()
max_m = e.groupBy('id').max('month').withColumnRenamed('max(month)','max_month')
e.join(e1, e.id==e1.id_1, 'leftouter')\
    .where(e.month >= e1.month_1)\
    .where((e.month-e1.month_1).between(0,2))\
    .groupBy('id','month')\
    .sum('salary_1')\
    .withColumnRenamed('sum(salary_1)', 'Salary')\
    .join(max_m, 'id','leftouter')\
    .where('month <> max_month')\
    .select('id', 'month', 'Salary')\
    .orderBy(['id','month'], ascending=[True,False])\
    .show()

root
 |-- id_1: integer (nullable = true)
 |-- month_1: integer (nullable = true)
 |-- salary_1: integer (nullable = true)

+---+-----+------+
| id|month|Salary|
+---+-----+------+
|  1|    7|    90|
|  1|    4|   130|
|  1|    3|    90|
|  1|    2|    50|
|  1|    1|    20|
|  2|    1|    20|
|  3|    3|   100|
|  3|    2|    40|
+---+-----+------+

